In [18]:
import random
import numpy as np
import networkx as nx 
from dijkstar import Graph, find_path
from ipywidgets import IntProgress 
from IPython.display import display

In [19]:
def generate_instance_matrix(nb_vertex, min_weight, max_weight, density):
  graph = np.ones((nb_vertex, nb_vertex)) * np.inf
  for i in range(nb_vertex - 1):
    for j in range(i+1, nb_vertex):
      if random.uniform(0, 1) < density:
        weight = random.randint(min_weight, max_weight)
        graph[i, j] = weight
        graph[j, i] = weight
  for i, successors in enumerate(graph):
    index_zeros = np.where(successors == np.inf)[0]
    index_zeros = np.delete(index_zeros, np.where(index_zeros == i))
    for j in range(len(index_zeros) - (nb_vertex - 3)):
      index = random.choice(index_zeros)
      index_zeros = np.delete(index_zeros, np.where(index_zeros == index))
      weight = random.randint(min_weight, max_weight)
      graph[i, index] = weight
      graph[index, i] = weight
  return graph

def convert_graph(graph):
  graph_dijkstra = Graph()
  for i in range(len(graph)):
    for j in range(len(graph)):
      graph_dijkstra.add_edge(i, j,graph[i][j])
  return graph_dijkstra

def get_distance_between_vertices(graph, i, j):
  if (graph[i][j] != np.inf):
    return graph[i][j], [i, j]
  else:
    if i != j:
      shortest_path = find_path(graph, i,j)
      return shortest_path.total_cost, shortest_path.nodes
    else:
      return 0, None

def get_complete_graph(graph, vertices_list):
  all_paths = {}
  converted_graph = convert_graph(graph)
  complete_graph = np.zeros((len(vertices_list), len(vertices_list)))
  bar = IntProgress(min=0, max=len(vertices_list), layout={"width" : "100%"})
  display(bar)
  for vertex in vertices_list:
    successors = np.delete(vertices_list, np.where(vertices_list == vertex))
    bar.value += 1
    for successor in successors:
      distance, path = get_distance_between_vertices(converted_graph, vertex, successor)
      i = np.where(vertices_list == vertex)[0][0]
      j = np.where(vertices_list == successor)[0][0]
      complete_graph[i][j] = distance
      all_paths[(i, j)] = path
  bar.close()
  return complete_graph, all_paths

def get_random_vertices_list(size, nb_vertex):
  random_vertices_list = []
  all_vertex = np.array(range(size))
  for _ in range(nb_vertex):
    value = random.choice(all_vertex)
    random_vertices_list.append(value)
    all_vertex = np.delete(all_vertex, np.where(all_vertex == value))
  return np.array(random_vertices_list)

def generate_interval():
  interval = random.choice(range(1,4))
  shift = random.choice(range(0, 11-interval))
  return (shift*60, (shift+interval)*60)

def generate_instance(nb_vertex_matrix, min_weight, max_weight, density, vertices_list):
  graph = generate_instance_matrix(nb_vertex_matrix, min_weight, max_weight, density)
  complete_graph, all_paths = get_complete_graph(graph, vertices_list)
  return graph, complete_graph, all_paths

def is_path_in_interval(path, complete_graph, interval, interval_vertex):
  weight = 0
  for i in range(np.where(np.array(path) == interval_vertex)[0][0]):
    weight += complete_graph[path[i]][path[i+1]]
  if weight >= interval[0] and weight <= interval[1]:
    return True
  else:
    return False

def generate_random_path(nb_vertex, start_vertex):
  path = np.arange(nb_vertex)
  path = np.delete(path, np.where(path == start_vertex))
  np.random.shuffle(path)
  path = np.append(path, start_vertex)
  path = np.insert(path, 0, start_vertex)
  return path

def get_random_vertex(random_vertex):
  return random.choice(random_vertex)

In [20]:
nb_vertex_matrix = 50
min_weight = 10
max_weight = 60
density = 0.5
nb_vertex_chosen = 4

#random generation of a list of vertex among all the vertex available
vertices_list = get_random_vertices_list(nb_vertex_matrix, nb_vertex_chosen)

start_vertex = get_random_vertex(vertices_list)
start_vertex_index = np.where(vertices_list == start_vertex)[0][0]

interval_vertex = get_random_vertex(vertices_list)
interval_vertex_index = np.where(vertices_list == interval_vertex)[0][0]

interval = generate_interval()

interval = (0,100000000)

# generation of the graph, the complete graph and the list of all paths
graph, complete_graph, all_paths =\
generate_instance(
  nb_vertex_matrix = nb_vertex_matrix,
  min_weight = min_weight,
  max_weight = max_weight,
  vertices_list = vertices_list,
  density = density
)

IntProgress(value=0, layout=Layout(width='100%'), max=4)

In [21]:
import pickle

data = {
  "graph": graph,
  "complete_graph": complete_graph,
  "all_paths": all_paths,
  "vertices_list": vertices_list,
  "start_vertex": start_vertex,
  "start_vertex_index": start_vertex_index,
  "interval_vertex": interval_vertex,
  "interval_vertex_index": interval_vertex_index,
  "interval": interval
}

with open('data.pickle', 'wb') as file:
    pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)